In [2]:
import json
import requests
from tensorflow import keras
from keras.layers import Activation, Dense, Dropout, LSTM
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import joblib

In [3]:
endpoint = "https://min-api.cryptocompare.com/data/histoday"
res = requests.get(endpoint + "?fsym=BTC&tsym=USD&limit=2000")
df = pd.DataFrame(json.loads(res.content)['Data'])
df.drop('conversionType', inplace=True, axis=1)
df.drop('conversionSymbol', inplace=True, axis=1)
df = df.set_index('time')
df.index = pd.to_datetime(df.index, unit = "s")

In [4]:
df.head()

,high,low,open,volumefrom,volumeto,close
time,,,,,,
2016-01-29,385.75,363.49,379.69,138455.81,51927041.84,380.29
2016-01-30,381.75,375.04,380.29,36179.60,13720656.84,377.76
2016-01-31,380.54,366.88,377.76,39019.71,14718855.75,369.84
2016-02-01,379.78,365.27,369.84,74346.01,27728088.91,372.18
2016-02-02,376.63,371.40,372.18,50981.07,19123252.48,373.92


In [5]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaled_data = scaler.fit_transform(df['close'].values.reshape(-1, 1))

In [6]:
pred_days = 30

In [7]:
x_train, y_train = [], []

for x in range(pred_days, len(scaled_data)):
    x_train.append(scaled_data[x-pred_days:x, 0])
    y_train.append(scaled_data[x, 0])

In [8]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [9]:
model = keras.Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error', metrics = [keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None)])
model.fit(x_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
62/62 [==============================] - 12s 64ms/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 2/30
62/62 [==============================] - 4s 64ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 3/30
62/62 [==============================] - 4s 63ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 4/30
62/62 [==============================] - 4s 61ms/step - loss: 0.0018 - mean_squared_error: 0.0018 0s - loss: 0.0019 - mean_squared_error
Epoch 5/30
62/62 [==============================] - 4s 64ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 6/30
62/62 [==============================] - 4s 64ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 7/30
62/62 [==============================] - 4s 64ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 8/30
62/62 [==============================] - 4s 62ms/step - loss: 0.0016 - mean_squared_error: 0.0016
Epoch 9/30
62/62 [==============================] - 4s 63ms/step - loss: 0.0016 - mean_s

In [10]:
model.save('model_1')

INFO:tensorflow:Assets written to: model_1\assets


INFO:tensorflow:Assets written to: model_1\assets


In [11]:
res = requests.get(endpoint + "?fsym=BTC&tsym=USD&limit=6")
test_data = pd.DataFrame(json.loads(res.content)['Data'])
test_data.drop('conversionType', inplace=True, axis=1)
test_data.drop('conversionSymbol', inplace=True, axis=1)
test_data = test_data.set_index('time')
test_data.index = pd.to_datetime(test_data.index, unit = "s")

In [12]:
total_dataset = pd.concat((df['close'], test_data['close']), axis=0)
total_dataset.index = pd.to_datetime(total_dataset.index)

In [13]:
model_inputs = total_dataset[len(total_dataset)-len(test_data)-pred_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)

In [14]:
x_test = []

for x in range(pred_days, len(model_inputs)):
    x_test.append(model_inputs[x-pred_days:x, 0])

In [15]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

In [16]:
prediction_prices

array([[31084.904],
       [31006.443],
       [31054.037],
       [31181.863],
       [31366.07 ],
       [31463.336],
       [31354.105]], dtype=float32)

In [32]:
from tensorflow import keras
loaded_model = keras.models.load_model('model_1')
predictions = loaded_model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions

array([[31084.904],
       [31006.443],
       [31054.037],
       [31181.863],
       [31366.07 ],
       [31463.336],
       [31354.105]], dtype=float32)

In [41]:
coins = ["Bitcoin", "Ethereum", "Dogecoin", "XRP", "Litecoin", "Stellar", "Tether"]
coin_symbols = ["BTC", "ETH", "DOGE", "XRP", "LTC", "XLM", "USDT"]

In [42]:
from datetime import datetime
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
day = datetime.today().weekday() + 1

In [66]:
predictions = {(days[(day+k)%7] if k != 0 else "Tmrw"):float(predictions[k][0]) for k in range(len(predictions))}

In [ ]:
predictions